In [1]:
import definitions as df
import data_manipulation as dm
import sql_def as sql
import os
import numpy as np
import matplotlib.pyplot as plt
import plots
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import math
import torch.nn.functional as F

Grabing the data from SQL dataset

In [2]:
path_to_database = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project\data"
path_to_code = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project"
os.chdir(path_to_database)
track_from_sql =  sql.data_from_sql("database.db", "SELECT * FROM TRACKMATEDATA LIMIT 7000")
os.chdir(path_to_code)
track_with_frame = df.separate_trajectories(track_from_sql)
trajectories = df.separate_data(track_with_frame, False)
len(trajectories)

473

Making Fake data

In [3]:
total_size = 100000
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 5

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)
trajectories

[[(345.0, 584.0),
  (348.5, 577.937822),
  (352.0, 571.875644),
  (355.5, 565.813467),
  (359.0, 559.751289)],
 [(884.0, 830.0),
  (890.913818, 828.904959),
  (897.827637, 827.809917),
  (904.741455, 826.714876),
  (911.655274, 825.619835)],
 [(138.0, 604.0),
  (138.0, 611.0),
  (138.0, 618.0),
  (138.0, 625.0),
  (138.0, 632.0)],
 [(743.0, 92.0),
  (743.587785, 92.809017),
  (744.175571, 93.618034),
  (744.763356, 94.427051),
  (745.351141, 95.236068)],
 [(909.0, 145.0),
  (904.854812, 147.795965),
  (900.709624, 150.591929),
  (896.564436, 153.387894),
  (892.419249, 156.183858)],
 [(408.0, 760.0),
  (407.164961, 765.941608),
  (406.329923, 771.883217),
  (405.494884, 777.824825),
  (404.659846, 783.766434)],
 [(77.0, 144.0),
  (77.156434, 144.987688),
  (77.312869, 145.975377),
  (77.469303, 146.963065),
  (77.625738, 147.950753)],
 [(943, 789), (946, 789), (949, 789), (952, 789), (955, 789)],
 [(214.0, 603.0),
  (205.13673, 604.562834),
  (196.27346, 606.125667),
  (187.410191, 607

In [4]:
norm_all_data = df.normalize_data(trajectories)
train, test = df.split_test_train(norm_all_data)
train_data, train_masked_point = df.mask_point_at_index(train, 2)
test_data, test_masked_point = df.mask_point_at_index(test, 2)

print(len(train_data))
print(len(test_masked_point))

90000
10000


In [5]:
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)

## DEBUG
print("src_data_tensor shape:", src_data_tensor.shape)
print("tgt_data_tensor shape:", tgt_data_tensor.shape)
print("src_masks_tensor shape:", src_masks_tensor.shape)

src_data_tensor shape: torch.Size([90000, 5, 2])
tgt_data_tensor shape: torch.Size([90000, 5, 2])
src_masks_tensor shape: torch.Size([90000, 5])


In [10]:
import model as md

input_dim = 2
hidden_dim = 64
output_dim = 2
rate = 0.01

model = md.EnhancedTrajectoryModel(input_dim, hidden_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)


In [11]:
dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
for epoch in range(3):  # Adjust the number of epochs as needed
        model.train()
        total_loss = 0

        for src, tgt, mask in dataloader:
            optimizer.zero_grad()
            output = model(src.view(-1, input_dim))
            output = output.view(src.shape)
            
            masked_output = output[mask]
            masked_tgt = tgt[mask]
            loss = loss_function(masked_output, masked_tgt)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 0.07332527999416222
Epoch 2, Loss: 0.07270082058116842
Epoch 3, Loss: 0.07270885399422568


In [13]:
t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
loss, accuracy = md.evaluate_model(model, t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor)
print(f"Test Loss: {loss}")
print(f"Accuracy (within threshold): {accuracy}%")

tensor(0.0725)
Test Loss: 0.07246355712413788
Accuracy (within threshold): 3.559999942779541%
